In [1]:
# Setup

# Characters are represented by 1-hot vectors of size 128
char_dim = 128

import numpy as np
import os
from collections import Counter
import unicodedata
import string
import gc
import matplotlib.pyplot as plt
import csv

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import LSTM
from torch import optim

In [59]:
np.random.seed(0)
songfile = open('songdata.csv', 'r')
songfile.readline()
csvreader = csv.reader(songfile, delimiter=',')
count = 0
while True:
    try:
        csvreader.__next__()
        count += 1
    except StopIteration:
        break
songfile.close()
print('count = %d' % count)

# randomly assign songs to train, val, and test files
perm = np.random.permutation(count)
songfile = open('songdata.csv', 'r')
songfile.readline()
csvreader = csv.reader(songfile, delimiter=',')

train_file = open('songdata_train.csv', 'w')
val_file = open('songdata_val.csv', 'w')
test_file = open('songdata_test.csv', 'w')
train_count = 0
val_count = 0
test_count = 0
train_writer = csv.writer(train_file)
val_writer = csv.writer(val_file)
test_writer = csv.writer(test_file)
for i, row in enumerate(csvreader):
    if perm[i] < count * 5 // 100:
        val_writer.writerow(row)
        val_count += 1
    elif perm[i] < count * 10 // 100:
        test_writer.writerow(row)
        test_count += 1
    else:
        train_writer.writerow(row)
        train_count += 1
train_file.close()
val_file.close()
test_file.close()
songfile.close()

print('training songs:   %d' % train_count)
print('validation songs: %d' % val_count)
print('test songs:       %d' % test_count)

count = 57471
training songs:   51724
validation songs: 2873
test songs:       2874


In [3]:
# converts a list of N strings of length <=T into a numpy array of 1-hot vectors
# input: list of length N; max length of any string in the list is T
# output size: (T, N, 128)
i128 = np.eye(128)
def char_to_ix(texts):
    T = max([len(text) for text in texts])
    ords = np.zeros((T, len(texts)), dtype=int)
    for n, text in enumerate(texts):
        ords[:len(text), n] = [ord(char) for char in text]
    return i128[ords]

# converts a list of N strings of length <=T into a numpy array of length (T, N).
# Zero-pads shorter strings.
def char_to_array(texts):
    T = max([len(text) for text in texts])
    result = np.zeros((T, len(texts)), dtype=int)
    for n, text in enumerate(texts):
        result[:len(text), n] = [ord(char) for char in text]
    return result
    #ords = np.array([[ord(char) for char in text] for text in texts], dtype=int)
    #return ords.transpose((1, 0))

In [8]:
class LyricsLSTM(nn.Module):
    def __init__(self, hidden_dim, num_stacks):
        super(LyricsLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(char_dim, hidden_dim, num_layers=num_stacks, dropout=0.0)
        
        # The linear layer that maps from hidden state space to character space
        self.hidden2char = nn.Linear(hidden_dim, char_dim)
        self.init_hidden_zeros(1)
    
    def init_hidden_zeros(self, minibatch_size):
        self.init_hidden(torch.zeros((self.lstm.num_layers, minibatch_size, self.hidden_dim)), torch.zeros((self.lstm.num_layers, minibatch_size, self.hidden_dim)))
    
    def init_hidden(self, h, c):
        self.hidden = (h, c)

    def forward(self, text):
        # text should be of size (T, N, char_dim)
        # returns character scores of size (T, N, char_dim)
        
        hs, self.hidden = self.lstm(text, self.hidden)
        char_space = self.hidden2char(hs)
        return char_space

In [66]:
def model_loss(model, loss_func, data_fname):
    model.lstm.eval()
    
    # set up csv reader
    songfile = open(data_fname, 'r')
    csvreader = csv.reader(songfile, delimiter=',')
    
    losses = []
    with torch.no_grad():
        model.init_hidden_zeros(1)
        for i, row in enumerate(csvreader):
            print('\rOn song %d of validation data' % i, end='')
            song = row[1] + '\n\n' + row[3]
            sequence_in = torch.tensor(char_to_ix([song]), dtype=torch.float)[:-1,:,:]
            sequence_out = torch.tensor(char_to_array([song]))[1:,:]
            char_scores = model(sequence_in)
            losses += [loss_func(char_scores.contiguous().view(-1, char_dim), sequence_out.contiguous().view(-1))]
    print()
    model.lstm.train()
    songfile.close()
    return np.mean(losses)

In [60]:
def train_loop(model, epochs, checkpoint_name=None, minibatch_size=16, optimizer=None):
    loss_func = torch.nn.CrossEntropyLoss()
    if optimizer == None:
        optimizer = optim.RMSprop(model.parameters())
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        print('on epoch %d' % epoch)
        
        # set up csv reader
        songfile = open('songdata_train.csv', 'r')
        csvreader = csv.reader(songfile, delimiter=',')
        
        epoch_finished = False
        minibatch_count = 0
        while not epoch_finished:
            # read minibatch from file
            # StopIteration means the minibatch is finished
            songs = []
            mb_data_ix = None
            mb_data_array = None
            try:
                for b in range(minibatch_size):
                    csvrow = csvreader.__next__()
                    songs += [csvrow[1] + '\n\n' + csvrow[3]]
                    mb_data_ix = torch.tensor(char_to_ix(songs), dtype=torch.float)
                    mb_data_array = torch.tensor(char_to_array(songs))
            except StopIteration:
                
                epoch_finished = True
                if mb_data_ix is None: # if we read no rows
                    break
            
            minibatch_count += 1
            print('\ron minibatch %d / %d' % (minibatch_count, (train_count - 1) // minibatch_size + 1), end='')
            model.zero_grad()

            sequence_in = mb_data_ix[:-1, :, :]
            sequence_out = mb_data_array[1:, :]

            # the last minibatch might have a different size if minibatch_size doesn't evenly divide the number of songs
            this_minibatch_size = sequence_in.shape[1]
            model.init_hidden_zeros(this_minibatch_size)

            char_scores = model(sequence_in)
            loss = loss_func(char_scores.contiguous().view(-1, char_dim), sequence_out.contiguous().view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
        print()
        songfile.close()
        #train_loss = model_loss(model, loss_func, train_data_ix, train_data_array)
        #val_loss = model_loss(model, loss_func, val_data_ix, val_data_array)
        #print('\ttraining loss = %f' % train_loss)
        #print('\tvalidation loss = %f' % val_loss)
        #train_losses += [train_loss]
        #val_losses += [val_loss]
        if checkpoint_name != None:
            torch.save(model.state_dict(), checkpoint_name + str(epoch))
    #return train_losses, val_losses

In [61]:
model = LyricsLSTM(64, 3)
train_loop(model, 50, checkpoint_name='lyrics_h64_l3_mb16_epoch')

on epoch 0
on minibatch 3233 / 3232
on epoch 1
on minibatch 3233 / 3232
on epoch 2
on minibatch 3233 / 3232
on epoch 3
on minibatch 3233 / 3232
on epoch 4
on minibatch 3233 / 3232
on epoch 5
on minibatch 1089 / 3232

KeyboardInterrupt: 

In [68]:
for epoch in range(5):
    model = LyricsLSTM(64, 3)
    model.load_state_dict(torch.load('lyrics_h64_l3_mb16' + str(epoch)))
    val_loss = model_loss(model, torch.nn.CrossEntropyLoss(), 'songdata_val.csv')
    print('for epoch %d, validation loss is %f' % (epoch, val_loss))

On song 2872 of validation data
for epoch 0, validation loss is 1.403628
On song 2872 of validation data
for epoch 1, validation loss is 1.352429
On song 2872 of validation data
for epoch 2, validation loss is 1.330566
On song 2872 of validation data
for epoch 3, validation loss is 1.315455
On song 2872 of validation data
for epoch 4, validation loss is nan


In [32]:
softmax = torch.nn.Softmax()
chars = range(128)

def sample_char(char_scores, temp):
    char_scores = softmax(char_scores / temp)
    char = np.random.choice(chars, p=char_scores.detach().numpy())
    while not chr(char) in string.printable and char != 0:
        char = np.random.choice(chars, p=char_scores.detach().numpy())
    return char

def sample(model, first_char, init_hidden, T, temp):
    model.init_hidden_zeros(1)
    result = first_char
    cur_char = ord(first_char)
    for t in range(T):
        one_hot_char = torch.tensor(i128[cur_char], dtype=torch.float).view(1, 1, -1)
        char_scores = model(one_hot_char)
        cur_char = sample_char(char_scores.view(-1), temp)
        if cur_char == 0:
            return result
        result += chr(cur_char)
    return result

sampled_song = sample(model, 'N', torch.zeros((1, 1, model.hidden_dim)), 500, 0.5)
print(sampled_song)

/home/alex/bin/anaconda3/envs/cs682/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


Never Once

It's the floor in the street  
  
[Chorus]  
  
Speak and the sea  
Love you on the trees  
We are the store  
  
You have you is gonna dance  
The street like you are  
  
It's the light of your heart  
  
I want you america  
  
She hear me see the time  
Can you do the curse  
  
Chotter of me  
  
I could hell you to keep the way  
Every day she hello  
I'm gonna get the wall the light  
I can't be the delies  
It's in the rain  
I got your same me  
  
I want to say the chance  

